In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import polars as pl
from freezegun import freeze_time

from src.activities import (
    get_activities_df,
    get_day_of_week_summaries,
    get_weekly_summaries,
)
from src.constants import COACH_ROLE
from src.auth_manager import get_strava_client
from src.email_manager import send_email, training_week_to_html
from src.supabase_client import (
    upsert_training_week_with_coaching,
    get_training_week_with_coaching,
    upsert_training_week_update,
)
from src.training_week_generation.training_week import generate_training_week

# Sunday Night Training Week Message

In [3]:
@freeze_time("2024-08-11 20:00:00")
def get_activities_df_wrapper(strava_client):
    return get_activities_df(strava_client)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

# activities setup
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df_wrapper(strava_client)

# gen training week pipeline
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)
training_week_with_coaching = generate_training_week(
    sysmsg_base=sysmsg_base,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries,
)

# save data to db and trigger email
upsert_training_week_with_coaching(
    athlete_id=athlete_id, training_week_with_coaching=training_week_with_coaching
)

athlete_id='98390356' token still valid until 2024-08-18 19:38:07+00:00


APIResponse(data=[{'id': 20, 'athlete_id': 98390356, 'training_week': '[{"day": "mon", "session_type": "easy run", "distance": 4.5, "notes": "Easy pace"}, {"day": "tues", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace"}, {"day": "wed", "session_type": "speed workout", "distance": 7.0, "notes": "Included intervals e.g. 4x1 mile at 10k pace with rest intervals"}, {"day": "thurs", "session_type": "easy run", "distance": 4.5, "notes": "Recovery pace after speed workout"}, {"day": "fri", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace"}, {"day": "sat", "session_type": "long run", "distance": 16.0, "notes": "Long run at a steady pace of 9m 43s per mile"}, {"day": "sun", "session_type": "rest day", "distance": 0.0, "notes": "Recovery day, complete rest"}]', 'typical_week_training_review': "Your training week reflects a balanced schedule with room for adjustments to optimize marathon preparation. You consistently run on Wednesdays and Saturdays, aligning with

In [4]:
send_email(
    subject="Training Schedule Just Dropped 🏃",
    html_content=training_week_to_html(training_week_with_coaching),
)

{'message_id': '<202408181427.29686375653@smtp-relay.mailin.fr>',
 'message_ids': None}

# Mid-Week Update

In [5]:
from src.types.activity_summary import ActivitySummary
from src.types.mid_week_analysis import MidWeekAnalysis
from src.llm import get_completion_json
from src.types.training_week import TrainingWeekWithPlanning


@freeze_time("2024-08-13 20:00:00")
def get_updated_training_week(
    mid_week_analysis: MidWeekAnalysis,
) -> TrainingWeekWithPlanning:
    client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
    sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

    msg = sysmsg_base + f"""\nThe original plan for this week targeted {mid_week_analysis.miles_target} miles. Your client has run {mid_week_analysis.miles_ran} miles so far (and they have {mid_week_analysis.miles_remaining} miles left to run this week), but the plan for the rest of the week includes {mid_week_analysis.future_miles_planned} miles. Adjust the plan accordingly given the following information:

    Here is your client's progress so far this week:
    {mid_week_analysis.activities}

    Here is the plan for the rest of the week:
    {mid_week_analysis.training_week_future}

    What changes need to be made so that your client runs {mid_week_analysis.miles_remaining} more miles this week?"""

    return get_completion_json(
            message=msg,
            response_model=TrainingWeekWithPlanning,
        )


@freeze_time("2024-08-13 20:00:00")
def mock_get_activities_df(strava_client, num_weeks=8):
    return get_activities_df(strava_client, num_weeks)

def get_activity_summaries(strava_client, num_weeks=8) -> pl.DataFrame:
    df = mock_get_activities_df(strava_client, num_weeks)
    concise_activities_df = df.with_columns(
        pl.col('start_date_local').apply(lambda x: x.strftime("%A, %B %d, %Y %I:%M %p")).alias('date_and_time'),
        pl.col('distance_in_miles').apply(lambda x: round(x, 2)),
        pl.col('elevation_gain_in_feet').apply(lambda x: round(x, 2)),
        pl.col('pace_minutes_per_mile').apply(lambda x: round(x, 2)),
    ).drop('id', 'name', 'day_of_week', 'week_of_year', 'year', 'moving_time_in_minutes', 'start_date_local')    
    return [ActivitySummary(**row) for row in concise_activities_df.rows(named=True)]

athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)

training_week_with_coaching = get_training_week_with_coaching(athlete_id)
current_weeks_activity_summaries = get_activity_summaries(strava_client, num_weeks=1)
mid_week_analysis = MidWeekAnalysis(activities=current_weeks_activity_summaries, training_week=training_week_with_coaching.training_week)

training_week_update_with_planning = get_updated_training_week(mid_week_analysis)

upsert_training_week_update(
    athlete_id=athlete_id,
    mid_week_analysis=mid_week_analysis,
    training_week_update_with_planning=training_week_update_with_planning,
)

athlete_id='98390356' token still valid until 2024-08-18 19:38:07+00:00


APIResponse(data=[{'id': 7, 'athlete_id': 98390356, 'activities': '[{"date_and_time": "Monday, August 12, 2024 06:06 AM", "distance_in_miles": 8.01, "elevation_gain_in_feet": 183.73, "pace_minutes_per_mile": 9.43}, {"date_and_time": "Tuesday, August 13, 2024 06:15 AM", "distance_in_miles": 4.01, "elevation_gain_in_feet": 52.49, "pace_minutes_per_mile": 10.64}]', 'training_week': '[{"day": "mon", "session_type": "easy run", "distance": 4.5, "notes": "Easy pace"}, {"day": "tues", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace"}, {"day": "wed", "session_type": "speed workout", "distance": 7.0, "notes": "Included intervals e.g. 4x1 mile at 10k pace with rest intervals"}, {"day": "thurs", "session_type": "easy run", "distance": 4.5, "notes": "Recovery pace after speed workout"}, {"day": "fri", "session_type": "easy run", "distance": 5.0, "notes": "Easy pace"}, {"day": "sat", "session_type": "long run", "distance": 16.0, "notes": "Long run at a steady pace of 9m 43s per mil

In [6]:
from src.email_manager import training_week_update_to_html

send_email(
    subject="Training Schedule Update 🏃",
    html_content=training_week_update_to_html(
        mid_week_analysis=mid_week_analysis, 
        training_week_update_with_planning=training_week_update_with_planning
    ),
)

{'message_id': '<202408181427.65417576067@smtp-relay.mailin.fr>',
 'message_ids': None}